<h1> Create New Design Ideas - Image Generation </h1>
<br>

Before starting, please make sure this notebook is using **conda_python3** kernel from the top right!

Run all the cells and inspect the output of each cell.

### Introduction

In this notebook, you will generate new design idea based on the existing product image from the catalog. For this feature, we will invoke the Bedrock API directly without Langchain using Bedrock's [InvokeModel](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/invoke_model.html) API. 

We will use Stability AI's [Stable Diffusion](https://aws.amazon.com/bedrock/stable-diffusion/) model from Bedrock that supports Text-To-Image and Image-To-Image transformations. We pass two optional prompt inputs to the model.

1. **Image prompt** will be used for creating new ideas from the existing image. For example: "*add animal prints to the shirt*". 
2. **Negative prompt** removes objects or styles in a way that may not be possible with image prompt alone. For example: "*bad quality*" or "*poorly rendered*". Stable diffusion model understands this prompt better than asking directly in the image prompt "*do not generate poorly rendered image*". 

![Image Generation](../images/image-generation.png)

### Install required dependencies

**Important:** You may see an error or a warning that "you may need to restart the kernel" from the following cell. **Ignore** and proceed with the next cells. 

In [ ]:
%pip install --quiet --no-build-isolation --upgrade \
    "boto3==1.28.63" \
    "awscli==1.29.63" \
    "botocore==1.31.63" 

<h3> Import required packages </h3>

In [ ]:
import json
import os
import sys
import boto3
import botocore

# For image operations
from PIL import Image
import base64
import io
import requests

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

<h3> Initialize Bedrock client </h3><br>

In [ ]:
 boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

<p>Create a function to convert an image into a base64 string since Stabile Diffusion Model expects the image to be in base64 string format</p>

In [ ]:
def image_to_base64(img) -> str:
    if isinstance(img, str):
        if os.path.isfile(img):
            with open(img, "rb") as f:
                return base64.b64encode(f.read()).decode("utf-8")
        else:
            raise FileNotFoundError(f"File {img} does not exist")
    elif isinstance(img, Image.Image):
        buffer = io.BytesIO()
        img.save(buffer, format="PNG")
        return base64.b64encode(buffer.getvalue()).decode("utf-8")
    else:
        raise ValueError(f"Expected str (filename) or PIL Image. Got {type(img)}")

### Read a sample product image

**Note:** We are using one of the images from the <a href="https://github.com/zalandoresearch/feidegger/tree/master">FEIDEGGER</a> dataset as an example. In the actual retail website, you will use the product image from the catalog. 


In [ ]:
image_url = "https://img01.ztat.net/article/spp-media-p1/3c8812d8b6233a55a5da06b19d780302/dc58460c157b426b817f13e7a2f087c5.jpg"

response = requests.get(image_url)
image = Image.open(io.BytesIO(response.content))

#### Resize product image to 512x512 and convert to base64 string format to comply with the requirements for Stable Diffusion. 

In [ ]:
resize = image.resize((512,512))
resize.show()

<h4> Convert this image to a base 64 string to pass into the model </h4>

In [ ]:
init_image_b64 = image_to_base64(resize)

### Input prompts 

These are the image prompts and the negative prompts we will be passing to the Stable Diffusion.

In [ ]:
# This prompt is used to generate new ideas from the existing image
change_prompt = "add floral prints to dress"

# Negative prompts that will be given -1.0 weight while generating new image
negative_prompts = ['poorly rendered',
                    'low quality',
                    'disfigured',
                    'disproportional']

### Compose request to pass to Stable Diffusion Model

This request includes our input prompts (image prompt, negative prompt) as well as the Stable Diffusion's [inference parameters](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-diffusion.html). **Note:** We describe these inference parameters in the **Explore** section of the workshop. 

In [ ]:
sd_request = json.dumps({
                    "text_prompts": (
                        [{"text": change_prompt, "weight": 1.0}]
                        + [{"text": negprompt, "weight": -1.0} for negprompt in negative_prompts]
                    ),
                    "cfg_scale": 10,
                    "init_image": init_image_b64,
                    "seed": 0,
                    "start_schedule": 0.5,
                    "steps": 30,
                    "style_preset": "photographic",
                    "image_strength":0.5,
                    "denoising_strength": 0.5
                })

### Call Bedrock to generate new image

Notice that we directly use the InvokeModel API from Bedrock. 

In [ ]:
response = boto3_bedrock.invoke_model(body=sd_request, modelId="stability.stable-diffusion-xl")

### Render the newly generated image

In [ ]:
response_body = json.loads(response.get('body').read())
genimage_b64_str = response_body["artifacts"][0].get("base64")
genimage = Image.open(io.BytesIO(base64.decodebytes(bytes(genimage_b64_str, "utf-8"))))
genimage.show()

<h3> You've successfully created new design ideas for a product with Amazon Bedrock!</h3>

Please stop the notebook kernel by selecting **Kernel -> Interrupt**.

#### Now, let's integrate this feature into our retail web application. Please go back to Workshop Studio and follow the instructions to build this feature using your Cloud9 IDE.